In [1]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Function Calling 與 Gemini API & Python SDK 介紹

<td style="text-align: center">
  <a href="https://colab.research.google.com/github/TWCkaijin/GDGC-Gemini-bootcamp/blob/main/function_calling.ipynb#scrollTo=VEqbX8OhE8y9">
    <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
  </a>
</td>

| | |
|-|-|
|Author(s) | [Kristopher Overholt](https://github.com/koverholt) [Holt Skinner](https://github.com/holtskinner) |

概述
YouTube 影片：AI + 你的程式碼：函式呼叫（Function Calling）

<a href="https://www.youtube.com/watch?v=NbAGbXr4DME&list=PLIivdWyY5sqLvGdVLJZh2EMax97_T-OIB" target="_blank"> <img src="https://img.youtube.com/vi/NbAGbXr4DME/maxresdefault.jpg" alt="AI + 你的程式碼：函式呼叫" width="500"> </a>

</br>

## Gemini
Gemini 是 Google DeepMind 開發的一系列生成式 AI 模型，專為多模態（multimodal）應用場景設計。


</br>

## 從 Gemini 呼叫函式
函式呼叫（Function Calling） 讓開發者可以在程式碼中描述一個函式，然後將該描述傳遞給語言模型進行請求。模型的回應會包含符合該描述的函式名稱以及應該使用的參數。

</br>

## 為什麼要使用函式呼叫？
想像一下，如果你請某人記錄重要資訊，但沒有提供表格或任何格式指引，對方可能會寫出一篇流暢的文章，但如果你需要從中提取特定的姓名、日期或數字，將會非常費力！同樣地，若沒有函式呼叫，想要從生成式文本模型獲得一致的結構化數據會是一大挑戰。你可能得不斷要求模型輸出 JSON 格式，但結果往往不穩定且令人沮喪。

這正是 Gemini 函式呼叫 的優勢所在。與其期待從自由格式的文本回應中拼湊出所需資訊，不如直接定義清楚的函式，並指定具體的參數和資料型別。這些函式定義相當於結構化的指引，能讓 Gemini 以可預測且可用的方式輸出結果。這樣一來，你就不需要再從文字回應中手動解析重要資訊了！

可以把它想像成教 Gemini 如何與你的應用程式對話。
需要從資料庫檢索資訊？定義一個 search_db 函式，並指定搜尋條件作為參數。
想要與天氣 API 整合？建立一個 get_weather 函式，並讓它接收地點作為輸入。
函式呼叫能夠橋接自然語言與結構化數據，讓 AI 更輕鬆地與外部系統互動！

## 任務目標
在本教學中，你將學習如何在 Vertex AI 中使用 Gemini API，並透過 Vertex AI SDK for Python 來使用 Gemini 2.0 Flash (gemini-2.0-flash) 模型進行函式呼叫（Function Calling）。

你將完成以下任務：

- 安裝 _Google Gen AI SDK for Python_
- 在 _Vertex AI_ 中使用 _Gemini API_ 與 _Gemini_ 模型互動：
- 在聊天會話（chat session）中使用 函式呼叫，回答使用者關於 Google Store 產品的問題
- 使用 _Function Calling_ 透過 地圖 API 進行地址地理編碼（geocoding）
- 使用 _Function Calling_ 在 原始日誌數據（raw logging data） 中進行實體擷取（entity extraction）

付費資源
本教學將使用 Google Cloud 中的 計費 功能：

- Vertex AI
請參閱 [Vertex AI](https://cloud.google.com/vertex-ai/pricing)價格 以了解詳細計費資訊，並使用 [費用估算](https://cloud.google.com/products/calculator/) 根據你的預計使用量估算成本。

## Getting Started


### 安裝 Google Gen AI SDK 套件


In [2]:
%pip install --upgrade --quiet google-genai

Note: you may need to restart the kernel to use updated packages.


### 重啟Colab執行個體

您剛剛安裝了一個套件，為確保他正確載入，我們將重啟執行個體

In [3]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

""" app = IPython.Application.instance()
app.kernel.do_shutdown(True) """

' app = IPython.Application.instance()\napp.kernel.do_shutdown(True) '

<div class="alert alert-block alert-warning">
<b>⚠️ 您的執行個體將會重啟，您會在左下角看到警示訊息 ⚠️</b>
</div>


### 驗證Colab環境 (Colab only)

In [4]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()
else: 
    from google.oauth2.service_account import Credentials
    SERVICE_ACCOUNT_FILE = 'cred.json'  # Path to your JSON file
    SCOPES = ['https://www.googleapis.com/auth/cloud-platform']  # IMPORTANT: Add the explicit scope
    creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)

### 設定 Google Cloud Platform 專案資訊


要開始使用 Vertex AI，你需要擁有一個 現有的 Google Cloud 專案，並[啟用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

你可以參考[設定專案與開發環境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)來了解更多詳細資訊。

In [5]:
import os

PROJECT_ID = "side-projcet-placeholder"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

from google import genai

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION, credentials=creds)

## 範例程式碼

### 選擇模型

想要了解更多關於Vertax AI 的 AI 模型 和 APIs, see [Google Models](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models#gemini-models) and [Model Garden](https://cloud.google.com/vertex-ai/generative-ai/docs/model-garden/explore-models).

In [6]:
MODEL_ID = "gemini-2.0-flash-001"  # @param {type: "string"}

### 載入套件


In [30]:
from IPython.display import Markdown, display
from google.genai.types import FunctionDeclaration, GenerateContentConfig, Part, Tool, Schema
import requests

### 聊天範例：在聊天會話中使用函式呼叫回答使用者關於 Google Store 的問題


在這個範例中，我們將使用 Gemini 的函式呼叫（Function Calling） 來建立一個聊天機器人，該機器人可以回答使用者關於 Google Store 產品的問題。

這樣的應用可以讓 AI 透過結構化的函式呼叫，而不是單純生成自由格式的文字，來提供更準確和一致的資訊。

In [ ]:
get_product_info = FunctionDeclaration(
    name="get_product_info",
    description="Get the stock amount and identifier for a given product",
    parameters={
        "type": "OBJECT",
        "properties": {
            "product_name": {"type": "STRING", "description": "Product name"}
        },
    },
)

get_store_location = FunctionDeclaration(
    name="get_store_location",
    description="Get the location of the closest store",
    parameters={
        "type": "OBJECT",
        "properties": {"location": {"type": "STRING", "description": "Location"}},
    },
)

place_order = FunctionDeclaration(
    name="place_order",
    description="Place an order",
    parameters={
        "type": "OBJECT",
        "properties": {
            "product": {"type": "STRING", "description": "Product name"},
            "address": {"type": "STRING", "description": "Shipping address"},
        },
    },
)

response=None description='Calculates the sum of two numbers.' name='add_numbers' parameters=Schema(min_items=None, example=None, property_ordering=None, pattern=None, minimum=None, default=None, any_of=None, max_length=None, title=None, min_length=None, min_properties=None, max_items=None, maximum=None, nullable=None, max_properties=None, type=<Type.OBJECT: 'OBJECT'>, description=None, enum=None, format=None, items=None, properties={'number1': Schema(min_items=None, example=None, property_ordering=None, pattern=None, minimum=None, default=None, any_of=None, max_length=None, title=None, min_length=None, min_properties=None, max_items=None, maximum=None, nullable=None, max_properties=None, type=<Type.NUMBER: 'NUMBER'>, description='The first number to add.', enum=None, format=None, items=None, properties=None, required=None), 'number2': Schema(min_items=None, example=None, property_ordering=None, pattern=None, minimum=None, default=None, any_of=None, max_length=None, title=None, min_len


請注意，函式參數需按照 [OpenAPI JSON Schema](https://spec.openapis.org/oas/v3.0.3#schemawr).
 格式 以 Python 字典（dictionary） 形式指定。

以下是定義工具（tool）的方法，讓 Gemini 模型 可以從 3 個函式 中進行選擇：


In [ ]:
retail_tool = Tool(
    function_declarations=[
        get_product_info,
        get_store_location,
        place_order,
    ],
)

現在，你可以在多輪對話（multi-turn chat session）中初始化 Gemini 模型，並啟用函式呼叫（Function Calling）。

在初始化聊天會話時，可以透過 `tools` 參數一次性指定可用的函式，這樣在後續的請求中就不需要每次重新傳遞這些函式設定。

In [43]:
chat = client.chats.create(
    model=MODEL_ID,
    config=GenerateContentConfig(
        temperature=0,
        tools=[retail_tool],
    ),
)

**注意事項**：
temperature 參數控制生成回應的隨機性：

較低的 temperature（如 0）：適合需要 **確定性（deterministic）** 的函式，例如傳遞固定格式的參數。
較高的 temperature：適合允許更具創意或多樣性參數值的函式，例如需要較自由輸入的應用場景。


當 temperature = 0 時，模型的輸出大多是確定性的，但仍可能有些許變化。

In [44]:
prompt = """
Do you have the Pixel 9 in stock?
"""

response = chat.send_message(prompt)
response.function_calls[0]

FunctionCall(id=None, args={'product_name': 'Pixel 9'}, name='get_product_info')

Gemini API 的回應包含一個結構化的資料物件，其中包括 Gemini 從可用函式中選擇的函式名稱以及對應的參數。

由於本教學的重點是提取函式參數並生成函式呼叫，因此你將使用模擬數據（mock data）來回傳合成回應給 Gemini 模型，而不是直接向 API 伺服器發送請求。（不用擔心！稍後的範例中，我們會進行實際的 API 呼叫。）

In [45]:
#在這裡，你可以使用你偏好的方法來發送 API 請求並獲取回應。
#在本範例中，我們將使用**模擬數據（synthetic data）**來模擬來自外部 API 的回應內容。

api_response = {"sku": "GA04834-US", "in_stock": "yes"}

在實際應用中，你會使用適當的客戶端函式庫或 REST API，對外部系統或資料庫執行函式呼叫。

現在，你可以將來自（模擬的）API 請求的回應傳遞給 Gemini 模型，並生成最終的使用者回應。

In [46]:
response = chat.send_message(
    Part.from_function_response(
        name="get_product_info",
        response={
            "content": api_response,
        },
    ),
)
display(Markdown(response.text))

Yes, we have the Pixel 9 in stock.


接下來，使用者可能會詢問在哪裡可以從附近的商店購買其他手機：

In [47]:
prompt = """
What about the Pixel 9 Pro XL? Is there a store in
Mountain View, CA that I can visit to try one out?
"""

response = chat.send_message(prompt)
response.function_calls

[FunctionCall(id=None, args={'product_name': 'Pixel 9 Pro XL'}, name='get_product_info'),
 FunctionCall(id=None, args={'location': 'Mountain View, CA'}, name='get_store_location')]

同樣地，你會收到一個包含結構化資料的回應，但這次請注意——這次不只是一個函式呼叫，而是兩個！

Gemini 模型 判斷需要同時呼叫 `get_product_info` 和 `get_store_location` 這兩個函式。

回頭看看幾個步驟前的對話提示，你會發現使用者詢問的不只是產品資訊，還有店鋪位置。

當多個函式被呼叫時
當定義了多個函式（或當模型預測需要對同一函式執行多次呼叫）時，Gemini 模型 可能會在同一回合內返回連續或並行的函式呼叫。

這些行為都是預料之內的，因為 Gemini 模型 會根據 **運行時推理（runtime prediction）** 來決定：

1. 應該呼叫哪些函式
2. 函式的執行順序（如果有相依性，則會依序執行）
3. 哪些函式可以並行執行，以便快速獲取足夠的資訊來生成自然語言回應

</br>

不用擔心！你可以重複相同的步驟，模擬 API 回應，來構造來自外部 API 的合成回應（synthetic payloads）。

In [15]:
# Here you can use your preferred method to make an API request and get a response.
# In this example, we'll use synthetic data to simulate a payload from an external API response.

product_info_api_response = {"sku": "GA08475-US", "in_stock": "yes"}
store_location_api_response = {
    "store": "2000 N Shoreline Blvd, Mountain View, CA 94043, US"
}

同樣地，你可以將來自（模擬的）API 請求的回應傳遞回 Gemini 模型。

In [16]:
response = chat.send_message(
    [
        Part.from_function_response(
            name="get_product_info",
            response={
                "content": product_info_api_response,
            },
        ),
        Part.from_function_response(
            name="get_store_location",
            response={
                "content": store_location_api_response,
            },
        ),
    ]
)
display(Markdown(response.text))

Yes, the Pixel 9 Pro XL is in stock. The store is located at 2000 N Shoreline Blvd, Mountain View, CA 94043, US.


### 做得很好！

在單次對話回合內，Gemini 模型 連續請求了 2 個函式呼叫，然後才返回自然語言摘要。

在實際應用中，如果你需要查詢庫存系統，然後再向店鋪位置資料庫、客戶管理系統或文件存儲系統發送 API 請求，這種模式將非常有用。

最後，使用者可能會請求訂購手機並將其配送到指定地址：

In [48]:
prompt = """
I'd like to order a Pixel 9 Pro XL and have it shipped to 1155 Borregas Ave, Sunnyvale, CA 94089.
"""

response = chat.send_message(prompt)
response.function_calls

[FunctionCall(id=None, args={'address': '1155 Borregas Ave, Sunnyvale, CA 94089', 'product': 'Pixel 9 Pro XL'}, name='place_order')]

太棒了！Gemini 模型 成功提取了使用者選擇的產品和他們的地址。現在，你可以呼叫 API 來完成訂單處理：

In [18]:
# This is where you would make an API request to return the status of their order.
# Use synthetic data to simulate a response payload from an external API.

order_api_response = {
    "payment_status": "paid",
    "order_number": 12345,
    "est_arrival": "2 days",
}

並將來自外部 API 呼叫的負載（payload）傳遞給 Gemini API，這樣 Gemini API 就會生成一個自然語言摘要，並回傳給最終使用者。

In [19]:
response = chat.send_message(
    Part.from_function_response(
        name="place_order",
        response={
            "content": order_api_response,
        },
    ),
)
display(Markdown(response.text))

OK. I have placed an order for a Pixel 9 Pro XL to be shipped to 1155 Borregas Ave, Sunnyvale, CA 94089. The order number is 12345 and it should arrive in 2 days. The payment status is paid.


恭喜你！

你成功地與 Gemini 模型 進行了多輪對話，使用函式呼叫、處理負載，並生成包含來自外部系統資訊的自然語言摘要

### 延伸-地址範例：使用自動函式呼叫（Automatic Function Calling）來透過地圖 API 進行地址地理編碼（geocode）

在這個範例中，你將定義一個接收多個參數的函式。然後，你將使用 Gemini API 的自動函式呼叫來執行實際的 API 呼叫，將地址轉換為緯度和經度座標。

首先，撰寫一個 Python 函式：

In [ ]:
def get_location(
    amenity: str | None = None,
    street: str | None = None,
    city: str | None = None,
    county: str | None = None,
    state: str | None = None,
    country: str | None = None,
    postalcode: str | None = None,
) -> list[dict]:
    """
    Get latitude and longitude for a given location.

    Args:
        amenity (str | None): Amenity or Point of interest.
        street (str | None): Street name.
        city (str | None): City name.
        county (str | None): County name.
        state (str | None): State name.
        country (str | None): Country name.
        postalcode (str | None): Postal code.

    Returns:
        list[dict]: A list of dictionaries with the latitude and longitude of the given location.
                    Returns an empty list if the location cannot be determined.
    """
    base_url = "https://nominatim.openstreetmap.org/search"
    params = {
        "amenity": amenity,
        "street": street,
        "city": city,
        "county": county,
        "state": state,
        "country": country,
        "postalcode": postalcode,
        "format": "json",
    }
    
    # Filter out None values from parameters
    params = {k: v for k, v in params.items() if v is not None}

    try:
        response = requests.get(base_url, params=params, headers={"User-Agent": "none"})
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"Error fetching location data: {e}")
        return []

在這個範例中，你會要求 Gemini 模型 將地址的組件提取到結構化資料物件中的特定欄位。這些欄位會被傳遞到你所定義的函式，然後回傳結果給 Gemini，以生成自然語言回應。

例如，發送一個包含地址的提示：

In [50]:
prompt = """
I want to get the coordinates for the following address:
1600 Amphitheatre Pkwy, Mountain View, CA 94043
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(tools=[get_location], temperature=0),
)
print(response.text)

d:\miniconda\lib\site-packages\pydantic\main.py:477: UserWarning: Pydantic serializer warnings:
  Expected `enum` but got `str` with value `'STRING'` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_json(
d:\miniconda\lib\site-packages\pydantic\main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `enum` but got `str` with value `'OBJECT'` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
d:\miniconda\lib\site-packages\pydantic\main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `enum` but got `str` with value `'STRING'` - serialized value may not be as expected
  Expected `enum` but got `str` with value `'STRING'` - serialized value may not be as expected
  Expected `enum` but got `str` with value `'STRING'` - serialized value may not be as expected
  Expected `enum` but got `str` with value `'STRING'` - serialized value may not be as expected
  Expected `enum` but got `str` with val

The coordinates for 1600 Amphitheatre Pkwy, Mountain View, CA 94043 are: latitude 37.42248575, longitude -122.08558456613565.



做得很好！你成功地定義了一個函式，並讓 Gemini 模型 用來從提示中提取相關的參數。接著，你進行了實際的 API 呼叫，獲取了指定位置的座標。

在這裡，我們使用了 [OpenStreetMap Nominatim API](https://nominatim.openstreetmap.org/ui/search.html) 來進行地址的地理編碼，以便讓本教學的步驟數量保持在合理範圍。如果你需要處理大量地址或地理位置數據，也可以使用 [Google Maps Geocoding API](https://developers.google.com/maps/documentation/geocoding)，或者任何提供 API 的地圖服務！

## Conclusions

你已完成探索 Google Gen AI Python SDK 的 `function calling` feature

深入探索 [更多資訊](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling).

## 範例實作 一

我們來時實作看看一個簡單的數學加法器

In [61]:
from IPython.display import Markdown, display
from google.genai.types import FunctionDeclaration, GenerateContentConfig, Part, Tool, Schema
import requests

In [76]:
def add_fn(a: float, b: float) -> float:
    return a + b

In [77]:
chat = client.chats.create(
    model=MODEL_ID,
    config=GenerateContentConfig(
        temperature=0,
        tools=[add_fn],
    ),
)

In [80]:
prompt = """
可以介紹給我幾間好吃的店家嗎?
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(tools=[add_fn], temperature=2),
)
print(response.text)

很抱歉，我目前無法提供美食店家資訊。



## 範例實作 二
來簡單的呼叫及整理API資料吧~

In [ ]:
from IPython.display import Markdown, display
from google.genai.types import FunctionDeclaration, GenerateContentConfig, Part, Tool, Schema
import requests

!curl -o bus.json https://raw.githubusercontent.com/nsysu-code-club/nsysu-bus/refs/heads/main/bus_zh.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  1275  100  1275    0     0   1486      0 --:--:-- --:--:-- --:--:--  1487


In [ ]:
chat = client.chats.create(
    model=MODEL_ID,
    config=GenerateContentConfig(
        temperature=0,
        tools=[add_fn],
    ),
)